In [1]:
import pandas as pd
import webbrowser
import os

In [60]:
#reading csv into dataframe
df = pd.read_csv('data/oct17-listings.csv', encoding='iso8859', index_col= 'row_id',usecols=['row_id','accommodates','bedrooms','bathrooms'])

In [61]:
#import model dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import mean_absolute_error,r2_score
from sklearn.externals import joblib

In [62]:
## load df dataframe into another dataframe where we can delete unwanted columns and still have big dataset as is
df.columns
df.shape

(8933, 3)

In [63]:
#finding out Nans
df.isnull().sum()

accommodates     0
bathrooms       31
bedrooms         8
dtype: int64

In [80]:
#replacing/filing Nans
df.bathrooms = df['bathrooms'].fillna(1)
df.bedrooms = df['bedrooms'].fillna(1)
df.bedrooms = df.bedrooms.astype(np.int64)
df.bathrooms = df.bathrooms.astype(np.int64)

In [217]:
X = df

In [218]:
y = pd.read_csv('data/oct17-listings.csv', encoding='iso8859', usecols=['price'])
y = df_target.replace({'\$':'',',':''},regex=True)
y.values.reshape(-1,1)
y.astype(np.float32) # use float 32 else, gradient boost model throws error for float 64
y.shape

(8933, 1)

In [219]:
# Split the data set in a training set (70%) and a test set (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6253, 3), (2680, 3), (6253, 1), (2680, 1))

In [93]:
#checking nans
X.isnull().sum()

accommodates    0
bathrooms       0
bedrooms        0
dtype: int64

In [94]:
X.bathrooms = X.bathrooms.fillna(1)
# X.bathrooms.isnull().sum()
X.bedrooms = X.bedrooms.fillna(1)
# X.bedrooms.isnull().sum()

In [95]:
# Fit regression model
model = ensemble.GradientBoostingRegressor(
    n_estimators=1000,
    learning_rate=0.5,
    max_depth=6,
    min_samples_leaf=5,
    max_features=0.2,
    loss='huber',
    random_state=2
)
model.fit(X_train, y_train)

/Users/manjeetchawla/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.5, loss='huber', max_depth=6,
             max_features=0.2, max_leaf_nodes=None,
             min_impurity_split=1e-07, min_samples_leaf=5,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, presort='auto', random_state=2,
             subsample=1.0, verbose=0, warm_start=False)

In [98]:
# Find the error rate on the training set
mse = mean_absolute_error(y_train, model.predict(X_train))
print("Training Set Mean Absolute Error: %.4f" % mse)

# Find the error rate on the test set
mse = mean_absolute_error(y_test, model.predict(X_test))
print("Test Set Mean Absolute Error: %.4f" % mse)

#r2 score
r2 = r2_score(y_test, model.predict(X_test))
print("r2 score: %.4f" % r2)

Training Set Mean Absolute Error: 102.9156
Test Set Mean Absolute Error: 108.6584
r2 score: 0.1834


In [99]:
#predicting results
preds = model.predict(X_test)
preds[:10]

array([ 213.9679313 ,  248.78055967,  248.78055967,  206.42020282,
        141.60467523,  141.60467523,  201.41603269,  141.60467523,
        141.60467523,  167.15080992])

In [215]:
# testing for custom values for input (accomodates, bedrooms, bathrooms)
listing = [
7, #accommodates
2, #bedrooms
2 #bathrooms
]

In [216]:
#putting it in another list
calculate_price=[listing]

#predicting new listing price
predict_potential = model.predict(calculate_price)
predict_potential

array([ 340.96170133])

## Model 2 

In [209]:
# Fit regression model
model2 = ensemble.GradientBoostingRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=2,
    min_samples_leaf=10,
    max_features=2,
    loss='ls',
    random_state=200
)

In [210]:
model2.fit(X_train, y_train)

/Users/manjeetchawla/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.01, loss='ls', max_depth=2, max_features=2,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=10, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=1000,
             presort='auto', random_state=200, subsample=1.0, verbose=0,
             warm_start=False)

In [211]:
# Find the error rate on the training set

mse = mean_absolute_error(y_train, model2.predict(X_train))
print("Training Set Mean Absolute Error: %.4f" % mse)

# Find the error rate on the test set
mse = mean_absolute_error(y_test, model2.predict(X_test))
print("Test Set Mean Absolute Error: %.4f" % mse)

#r2 score
pred2 = model2.predict(X_test)
r2 = r2_score(y_test, pred2)
print("r2 score: %.4f" % r2)

Training Set Mean Absolute Error: 127.3404
Test Set Mean Absolute Error: 121.8507
r2 score: 0.2542


## <font color='skyblue'>Model 3 - using amenities in Amenities notebook</font>

## Transfering model to pickle file so it can be accessed from other files

In [212]:
joblib.dump(model2, 'potential_listing_price.pkl')

['potential_listing_price.pkl']

## GB Model using <font color='green'>Neighborhood feature</font>

In [271]:
## creating dummies for neighborhood column from main df and concatinating with X

nb = pd.read_csv('data/oct17-listings.csv', encoding='iso8859', usecols=['neighbourhood_cleansed'])
nb.head()

,neighbourhood_cleansed
0,Seacliff
1,Seacliff
2,Seacliff
3,Seacliff
4,Seacliff


In [272]:
nb= pd.get_dummies(nb, dummy_na=False, drop_first=True)
nb.head()

,neighbourhood_cleansed_Bernal Heights,neighbourhood_cleansed_Castro/Upper Market,neighbourhood_cleansed_Chinatown,neighbourhood_cleansed_Crocker Amazon,neighbourhood_cleansed_Diamond Heights,neighbourhood_cleansed_Downtown/Civic Center,neighbourhood_cleansed_Excelsior,neighbourhood_cleansed_Financial District,neighbourhood_cleansed_Glen Park,neighbourhood_cleansed_Golden Gate Park,...,neighbourhood_cleansed_Presidio,neighbourhood_cleansed_Presidio Heights,neighbourhood_cleansed_Russian Hill,neighbourhood_cleansed_Seacliff,neighbourhood_cleansed_South of Market,neighbourhood_cleansed_Treasure Island/YBI,neighbourhood_cleansed_Twin Peaks,neighbourhood_cleansed_Visitacion Valley,neighbourhood_cleansed_West of Twin Peaks,neighbourhood_cleansed_Western Addition
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [273]:
nb = nb.astype(np.int64)

In [274]:
df2 = pd.concat([df,nb], axis=1)

In [275]:
df2.head()

,accommodates,bathrooms,bedrooms,neighbourhood_cleansed_Bernal Heights,neighbourhood_cleansed_Castro/Upper Market,neighbourhood_cleansed_Chinatown,neighbourhood_cleansed_Crocker Amazon,neighbourhood_cleansed_Diamond Heights,neighbourhood_cleansed_Downtown/Civic Center,neighbourhood_cleansed_Excelsior,...,neighbourhood_cleansed_Presidio,neighbourhood_cleansed_Presidio Heights,neighbourhood_cleansed_Russian Hill,neighbourhood_cleansed_Seacliff,neighbourhood_cleansed_South of Market,neighbourhood_cleansed_Treasure Island/YBI,neighbourhood_cleansed_Twin Peaks,neighbourhood_cleansed_Visitacion Valley,neighbourhood_cleansed_West of Twin Peaks,neighbourhood_cleansed_Western Addition
0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [277]:
df2 = df2.drop(df2.index[[0]])

In [278]:
df2.head()

,accommodates,bathrooms,bedrooms,neighbourhood_cleansed_Bernal Heights,neighbourhood_cleansed_Castro/Upper Market,neighbourhood_cleansed_Chinatown,neighbourhood_cleansed_Crocker Amazon,neighbourhood_cleansed_Diamond Heights,neighbourhood_cleansed_Downtown/Civic Center,neighbourhood_cleansed_Excelsior,...,neighbourhood_cleansed_Presidio,neighbourhood_cleansed_Presidio Heights,neighbourhood_cleansed_Russian Hill,neighbourhood_cleansed_Seacliff,neighbourhood_cleansed_South of Market,neighbourhood_cleansed_Treasure Island/YBI,neighbourhood_cleansed_Twin Peaks,neighbourhood_cleansed_Visitacion Valley,neighbourhood_cleansed_West of Twin Peaks,neighbourhood_cleansed_Western Addition
1,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
5,7.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [299]:
df2 = df2.fillna(0)

In [300]:
df2.isnull().sum()

accommodates                                    0
bathrooms                                       0
bedrooms                                        0
neighbourhood_cleansed_Bernal Heights           0
neighbourhood_cleansed_Castro/Upper Market      0
neighbourhood_cleansed_Chinatown                0
neighbourhood_cleansed_Crocker Amazon           0
neighbourhood_cleansed_Diamond Heights          0
neighbourhood_cleansed_Downtown/Civic Center    0
neighbourhood_cleansed_Excelsior                0
neighbourhood_cleansed_Financial District       0
neighbourhood_cleansed_Glen Park                0
neighbourhood_cleansed_Golden Gate Park         0
neighbourhood_cleansed_Haight Ashbury           0
neighbourhood_cleansed_Inner Richmond           0
neighbourhood_cleansed_Inner Sunset             0
neighbourhood_cleansed_Lakeshore                0
neighbourhood_cleansed_Marina                   0
neighbourhood_cleansed_Mission                  0
neighbourhood_cleansed_Nob Hill                 0


In [301]:
F = df2
t =y
t.shape, F.shape

((8933, 1), (8933, 39))

In [302]:
# Split the data set in a training set (70%) and a test set (30%)
F_train, F_test, t_train, t_test = train_test_split(F, t, test_size=0.3, random_state=0)
F_train.shape, F_test.shape, t_train.shape, t_test.shape

((6253, 39), (2680, 39), (6253, 1), (2680, 1))

In [355]:
# Fit regression model
model5 = ensemble.GradientBoostingRegressor(
    n_estimators=3000,
    learning_rate=0.15,
    max_depth=5,
    min_samples_leaf=30,
    max_features=1,
    loss='ls',
    random_state=200
) #r2score: 0.2866

# model5 = ensemble.GradientBoostingRegressor(
#     n_estimators=3000,
#     learning_rate=0.1,
#     max_depth=6,
#     min_samples_leaf=10,
#     max_features=1,
#     loss='ls',
#     random_state=200
# ) #r2 score: 0.2575

# model5 = ensemble.GradientBoostingRegressor(
#     n_estimators=1000,
#     learning_rate=0.01,
#     max_depth=2,
#     min_samples_leaf=10,
#     max_features=2,
#     loss='ls',
#     random_state=200
# ) # r2score : 0.2926

In [356]:
#fit the model
model5.fit(F_train,t_train)

/Users/manjeetchawla/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.15, loss='ls', max_depth=5, max_features=1,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=30, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             presort='auto', random_state=200, subsample=1.0, verbose=0,
             warm_start=False)

In [357]:
## Find the error rate on the training set

mse = mean_absolute_error(t_train, model5.predict(F_train))
print("Training Set Mean Absolute Error: %.4f" % mse)

# Find the error rate on the test set
mse = mean_absolute_error(t_test, model5.predict(F_test))
print("Test Set Mean Absolute Error: %.4f" % mse)

#r2 score
pred5 = model5.predict(F_test)
r5 = r2_score(t_test, pred5)
print("r2 score: %.4f" % r5)

Training Set Mean Absolute Error: 124.5150
Test Set Mean Absolute Error: 121.4072
r2 score: 0.3023


In [372]:
features = [[
5, #accommodates                                  
2, #bathrooms                                      
2, #bedrooms                                       
0, #neighbourhood_cleansed_Bernal Heights          
0, #neighbourhood_cleansed_Castro/Upper Market     
0, #neighbourhood_cleansed_Chinatown               
0, #neighbourhood_cleansed_Crocker Amazon          
0, #neighbourhood_cleansed_Diamond Heights         
0, #neighbourhood_cleansed_Downtown/Civic Center   
0, #neighbourhood_cleansed_Excelsior               
0, #neighbourhood_cleansed_Financial District      
0, #neighbourhood_cleansed_Glen Park               
0, #neighbourhood_cleansed_Golden Gate Park        
0, #neighbourhood_cleansed_Haight Ashbury          
0, #neighbourhood_cleansed_Inner Richmond          
0, #neighbourhood_cleansed_Inner Sunset            
0, #neighbourhood_cleansed_Lakeshore               
0, #neighbourhood_cleansed_Marina                  
0, #neighbourhood_cleansed_Mission                  
0, #neighbourhood_cleansed_Nob Hill                 
0, #neighbourhood_cleansed_Noe Valley               
0, #neighbourhood_cleansed_North Beach              
0, #neighbourhood_cleansed_Ocean View               
0, #neighbourhood_cleansed_Outer Mission            
0, #neighbourhood_cleansed_Outer Richmond           
0, #neighbourhood_cleansed_Outer Sunset             
0, #neighbourhood_cleansed_Pacific Heights          
0, #neighbourhood_cleansed_Parkside                 
0, #neighbourhood_cleansed_Potrero Hill             
0, #neighbourhood_cleansed_Presidio                 
1, #neighbourhood_cleansed_Presidio Heights         
0, #neighbourhood_cleansed_Russian Hill             
0, #neighbourhood_cleansed_Seacliff                 
0, #neighbourhood_cleansed_South of Market          
0, #neighbourhood_cleansed_Treasure Island/YBI      
0, #neighbourhood_cleansed_Twin Peaks               
0, #neighbourhood_cleansed_Visitacion Valley        
0, #neighbourhood_cleansed_West of Twin Peaks       
0 #neighbourhood_cleansed_Western Addition
]]

In [400]:
#predict price with given features
estimate = model5.predict(features)
print('Your airbnb listing price should be approx: $%.2f' % estimate[0])

Your airbnb listing price should be approx: $406.68


In [403]:
# saving model5 to pickle file to be used for predictions
joblib.dump(model5, 'predict_price_bd_bh_nb.pkl')

['predict_price_bd_bh_nb.pkl']